In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 31 03:00:36 2020

@author: hp
"""

import cv2
import numpy as np
import math
from Proctoring-AI.face_detector import get_face_detector, find_faces
from Proctoring-AI.face_landmarks import get_landmark_model, detect_marks
from datetime import datetime
import time
import pandas as pd

def get_2d_points(img, rotation_vector, translation_vector, camera_matrix, val):
    """Return the 3D points present as 2D for making annotation box"""
    point_3d = []
    dist_coeffs = np.zeros((4,1))
    rear_size = val[0]
    rear_depth = val[1]
    point_3d.append((-rear_size, -rear_size, rear_depth))
    point_3d.append((-rear_size, rear_size, rear_depth))
    point_3d.append((rear_size, rear_size, rear_depth))
    point_3d.append((rear_size, -rear_size, rear_depth))
    point_3d.append((-rear_size, -rear_size, rear_depth))
    
    front_size = val[2]
    front_depth = val[3]
    point_3d.append((-front_size, -front_size, front_depth))
    point_3d.append((-front_size, front_size, front_depth))
    point_3d.append((front_size, front_size, front_depth))
    point_3d.append((front_size, -front_size, front_depth))
    point_3d.append((-front_size, -front_size, front_depth))
    point_3d = np.array(point_3d, dtype=np.float).reshape(-1, 3)
    
    # Map to 2d img points
    (point_2d, _) = cv2.projectPoints(point_3d,
                                      rotation_vector,
                                      translation_vector,
                                      camera_matrix,
                                      dist_coeffs)
    point_2d = np.int32(point_2d.reshape(-1, 2))
    return point_2d

def draw_annotation_box(img, rotation_vector, translation_vector, camera_matrix,
                        rear_size=300, rear_depth=0, front_size=500, front_depth=400,
                        color=(255, 255, 0), line_width=2):
    """
    Draw a 3D anotation box on the face for head pose estimation

    Parameters
    ----------
    img : np.unit8
        Original Image.
    rotation_vector : Array of float64
        Rotation Vector obtained from cv2.solvePnP
    translation_vector : Array of float64
        Translation Vector obtained from cv2.solvePnP
    camera_matrix : Array of float64
        The camera matrix
    rear_size : int, optional
        Size of rear box. The default is 300.
    rear_depth : int, optional
        The default is 0.
    front_size : int, optional
        Size of front box. The default is 500.
    front_depth : int, optional
        Front depth. The default is 400.
    color : tuple, optional
        The color with which to draw annotation box. The default is (255, 255, 0).
    line_width : int, optional
        line width of lines drawn. The default is 2.

    Returns
    -------
    None.

    """
    
    rear_size = 1
    rear_depth = 0
    front_size = img.shape[1]
    front_depth = front_size*2
    val = [rear_size, rear_depth, front_size, front_depth]
    point_2d = get_2d_points(img, rotation_vector, translation_vector, camera_matrix, val)
    # # Draw all the lines
    cv2.polylines(img, [point_2d], True, color, line_width, cv2.LINE_AA)
    cv2.line(img, tuple(point_2d[1]), tuple(
        point_2d[6]), color, line_width, cv2.LINE_AA)
    cv2.line(img, tuple(point_2d[2]), tuple(
        point_2d[7]), color, line_width, cv2.LINE_AA)
    cv2.line(img, tuple(point_2d[3]), tuple(
        point_2d[8]), color, line_width, cv2.LINE_AA)
    
    
def head_pose_points(img, rotation_vector, translation_vector, camera_matrix):
    """
    Get the points to estimate head pose sideways    

    Parameters
    ----------
    img : np.unit8
        Original Image.
    rotation_vector : Array of float64
        Rotation Vector obtained from cv2.solvePnP
    translation_vector : Array of float64
        Translation Vector obtained from cv2.solvePnP
    camera_matrix : Array of float64
        The camera matrix

    Returns
    -------
    (x, y) : tuple
        Coordinates of line to estimate head pose

    """
    rear_size = 1
    rear_depth = 0
    front_size = img.shape[1]
    front_depth = front_size*2
    val = [rear_size, rear_depth, front_size, front_depth]
    point_2d = get_2d_points(img, rotation_vector, translation_vector, camera_matrix, val)
    y = (point_2d[5] + point_2d[8])//2
    x = point_2d[2]
    
    return (x, y)

import os

directory = os.fsencode('C:/Users/Catherine/RPL_Language/collab-SLA/data_evaluation/Videos/')
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    video = 'C:/Users/Catherine/RPL_Language/collab-SLA/data_evaluation/Videos/'+filename
    
    face_model = get_face_detector()
    landmark_model = get_landmark_model()
    cap = cv2.VideoCapture(video)
    #cap = cv2.VideoCapture(0)
    ret, img = cap.read()
    size = img.shape
    font = cv2.FONT_HERSHEY_SIMPLEX 
    # 3D model points.
    model_points = np.array([
                                (0.0, 0.0, 0.0),             # Nose tip
                                (0.0, -330.0, -65.0),        # Chin
                                (-225.0, 170.0, -135.0),     # Left eye left corner
                                (225.0, 170.0, -135.0),      # Right eye right corne
                                (-150.0, -150.0, -125.0),    # Left Mouth corner
                                (150.0, -150.0, -125.0)      # Right mouth corner
                            ])

    # Camera internals
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )

    left = []
    right = []
    dateTimeObj = datetime.now()
    start_time = dateTimeObj.second
    fps = cap.get(cv2.CAP_PROP_FPS)
    totalNoFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT);
    durationInSeconds = float(totalNoFrames) / float(fps)
    # cap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    # cap.get(cv2.CAP_PROP_POS_MSEC)
    # print(cap.get(cv2.CAP_PROP_POS_MSEC))
    # print(durationInSeconds)\
    left_pos = [0,0]
    right_pos = [0,0]
    old_time = cap.get(cv2.CAP_PROP_POS_MSEC)/1000
    while True:
        ret, img = cap.read()
        if ret == True:
            new_time = cap.get(cv2.CAP_PROP_POS_MSEC)/1000
            if new_time - old_time >= 0.1:
                faces = find_faces(img, face_model)
                for face in faces:
                    if face[0] > 1000:
                        person = 'left'
                    else:
                        person = 'right'
                    marks = detect_marks(img, landmark_model, face)
                    # mark_detector.draw_marks(img, marks, color=(0, 255, 0))
                    image_points = np.array([
                                            marks[30],     # Nose tip
                                            marks[8],     # Chin
                                            marks[36],     # Left eye left corner
                                            marks[45],     # Right eye right corne
                                            marks[48],     # Left Mouth corner
                                            marks[54]      # Right mouth corner
                                        ], dtype="double")
                    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
                    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_UPNP)


                    # Project a 3D point (0, 0, 1000.0) onto the image plane.
                    # We use this to draw a line sticking out of the nose

                    (nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)

                    for p in image_points:
                        cv2.circle(img, (int(p[0]), int(p[1])), 3, (0,0,255), -1)


                    p1 = ( int(image_points[0][0]), int(image_points[0][1]))
                    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
                    x1, x2 = head_pose_points(img, rotation_vector, translation_vector, camera_matrix)

                    cv2.line(img, p1, p2, (0, 255, 255), 2)
                    cv2.line(img, tuple(x1), tuple(x2), (255, 255, 0), 2)
                    # for (x, y) in marks:
                    #     cv2.circle(img, (x, y), 4, (255, 255, 0), -1)
                    # cv2.putText(img, str(p1), p1, font, 1, (0, 255, 255), 1)
                    try:
                        m = (p2[1] - p1[1])/(p2[0] - p1[0])
                        ang1 = int(math.degrees(math.atan(m)))
                    except:
                        ang1 = 90

                    try:
                        m = (x2[1] - x1[1])/(x2[0] - x1[0])
                        ang2 = int(math.degrees(math.atan(-1/m)))
                    except:
                        ang2 = 90

                        # print('div by zero error')
                    if ang1 >= 48:
                        #print(person,face,'Head down')
                        cv2.putText(img, 'Head down', (30, 30), font, 2, (255, 255, 128), 3)
                    elif ang1 <= -48:
                        #print(person,face,'Head up')
                        cv2.putText(img, 'Head up', (30, 30), font, 2, (255, 255, 128), 3)

                    if ang2 >= 48:
                        #print(person,face,'Head right')
                        cv2.putText(img, 'Head right', (90, 30), font, 2, (255, 255, 128), 3)
                    elif ang2 <= -48:
                        #print(person,face,'Head left')
                        cv2.putText(img, 'Head left', (90, 30), font, 2, (255, 255, 128), 3)
                    if person == 'left':
                        left_pos = [ang1,ang2]
                    if person == 'right':
                        right_pos = [ang1,ang2]
                    cv2.putText(img, str(ang1), tuple(p1), font, 2, (128, 255, 255), 3)
                    cv2.putText(img, str(ang2), tuple(x1), font, 2, (255, 255, 128), 3)
                left.append([new_time,left_pos[0],left_pos[1]])
                right.append([new_time,right_pos[0],right_pos[1]])
                old_time = new_time
                cv2.imshow('img', img)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
        else:
            break
    cv2.destroyAllWindows()
    cap.release()
    leftdf = pd.DataFrame(left)
    leftdf.columns = ['time_elapsed (s)', 'left angle vertical', 'left angle horizontal']
    rightdf = pd.DataFrame(right)
    rightdf.columns = ['time_elapsed (s)', 'right angle vertical', 'right angle horizontal']
    combined_results = leftdf
    combined_results['right angle vertical'] = rightdf['right angle vertical']
    combined_results['right angle horizontal'] = rightdf['right angle horizontal']
    combined_results.to_csv('participant_csvs/'+filename[:-4]+'.csv')
    print('saved'+filename)

saved10-01-2020-Furhat__100101.mp4


C:\Users\Catherine\Anaconda3\lib\site-packages\ipykernel_launcher.py:228: RuntimeWarning: divide by zero encountered in int_scalars


saved10-01-2020-Furhat__9495.mp4
saved10-01-2020-Furhat__9697.mp4
saved10-01-2020-Furhat__9899.mp4
saved10-02-2020-Furhat__102103.mp4
saved10-02-2020-Furhat__104105.mp4
saved10-02-2020-Furhat__106107 (1).mp4
saved10-02-2020-Furhat__108109.mp4
saved9-18-2020-Furhat__4041.mp4
saved9-21-2020-Furhat__4243.mp4
saved9-21-2020-Furhat__4445.mp4
saved9-23-2020-Furhat__4647_NoReleaseRight.mp4
saved9-23-2020-Furhat__4849.mp4
saved9-23-2020-Furhat__5051.mp4
saved9-23-2020-Furhat__5253.mp4
saved9-23-2020-Furhat__5455.mp4
saved9-23-2020-Furhat__5657.mp4
saved9-24-2020-Furhat__5859.mp4
saved9-24-2020-Furhat__6061_NoReleaseRight.mp4
saved9-24-2020-Furhat__6263.mp4
saved9-24-2020-Furhat__6465.mp4
saved9-25-2020-Furhat__6667_NoReleaseRight.mp4
saved9-25-2020-Furhat__6869.mp4
saved9-25-2020-Furhat__7071.mp4
saved9-25-2020-Furhat__7273.mp4
saved9-26-2020-Furhat__7475.mp4
saved9-28-2020-Furhat__7677_NoReleaseRight.mp4
saved9-28-2020-Furhat__7879.mp4
saved9-28-2020-Furhat__8081.mp4
saved9-28-2020-Furhat__82